In [1]:
# data analysis stack
import pandas as pd
import numpy as np

# data visualization stack
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set() # set seaborn as default style

# data pre-processing stack
from sklearn.preprocessing import (
    StandardScaler,
    OneHotEncoder,
    PolynomialFeatures
)
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

#machine learning stack
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression


from sklearn.model_selection import GridSearchCV

# miscellaneous
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
bike = pd.read_csv("./bike_train.csv", index_col=0, parse_dates=True)
bike.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
datetime,,,,,,,,,,,
2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [3]:
bike["log_casual"]=np.log1p(bike["casual"])
bike["log_registered"]=np.log1p(bike["registered"])

In [4]:
def yoda(df):
    df["year"] = df.index.year
    df["month"] = df.index.month
    df["hour"] = df.index.hour
    df["weather"].replace([4], [3], inplace = True)
    return df

In [5]:
yoda(bike)
bike

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,log_casual,log_registered,year,month,hour
datetime,,,,,,,,,,,,,,,,
2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16,1.386294,2.639057,2011,1,0
2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40,2.197225,3.496508,2011,1,1
2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32,1.791759,3.332205,2011,1,2
2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13,1.386294,2.397895,2011,1,3
2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1,0.000000,0.693147,2011,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,336,2.079442,5.799093,2012,12,19
2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,241,2.397895,5.446737,2012,12,20
2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,168,1.609438,5.105945,2012,12,21


In [ ]:
bike=bike.drop(bike[bike["weather"]==4].index)
bike.weather.value_counts()

In [6]:
def month_index(year, month):
    return (year-2011)*12 + month
    
bike['month_idx'] = month_index(bike["year"], bike["month"])

In [7]:
bike['day_of_week'] = bike.index.weekday + 1
bike.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,log_casual,log_registered,year,month,hour,month_idx,day_of_week
datetime,,,,,,,,,,,,,,,,,,
2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,1.386294,2.639057,2011,1,0,1,6
2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2.197225,3.496508,2011,1,1,1,6
2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,1.791759,3.332205,2011,1,2,1,6
2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,1.386294,2.397895,2011,1,3,1,6
2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,0.000000,0.693147,2011,1,4,1,6


In [8]:
"""# Working days is completely the same with weeknds. No need to create one.
def weekend_trans(weekday):
    if weekday in ["Saturday", "Sunday"]:
        return 1
    else:
        return 0
    
bike['weekends'] = bike["dow_label"].apply(weekend_trans)"""
 

'# Working days is completely the same with weeknds. No need to create one.\ndef weekend_trans(weekday):\n    if weekday in ["Saturday", "Sunday"]:\n        return 1\n    else:\n        return 0\n    \nbike[\'weekends\'] = bike["dow_label"].apply(weekend_trans)'

**CASUAL USERS**

In [9]:
train,test = train_test_split(bike, test_size=0.2, random_state=101)

In [10]:
# we need to delete either the "temp" or the "atemp" because they are highly corelated with each other.
numerical_features = [
     'temp',
     #'atemp', ## temp was better than atemp. Training score is 2 points more 
     'humidity', ## add alot like 3 points
     'windspeed', # added some more power
     'month_idx',
     'hour' 
]

categorical_features = [
    #'season', Adding season does not add much. Mostly weather matters.
    'holiday',
    'workingday',
    'day_of_week',
    'weather' # our as a categorical training score is 78.
]

features = numerical_features + categorical_features

target_variable = 'log_casual' # for registered it it aroun 20 but casual gets 45 with these features except atemp.


In [11]:
X_casual_train,y_casual_train = train[features], train[target_variable]

In [12]:
# scaling and polynomial features
numerical_transformer = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('polynomial', PolynomialFeatures())
    ]
)

In [13]:
# one-hot encoding
categorical_transformer = Pipeline(
    steps=[
        ('ohe', OneHotEncoder(drop='first'))
    ]
)

In [14]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_transformer, numerical_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)

In [15]:
estimator = Pipeline(
    steps=[
        ('preprocessor', preprocessor),   # preprocessing step
        ('lasso', Lasso()) # lasso regression
    ]
)

In [16]:
param_grid = {
    'preprocessor__num__polynomial__degree': [4],
    'preprocessor__num__polynomial__interaction_only': [False,True],
    'lasso__alpha': [100.,10.,1.,0.1,0.01],
    'lasso__max_iter': [5_000, 10_000,20_000]
}

In [17]:
gscv = GridSearchCV(
    estimator=estimator,
    param_grid=param_grid,
    scoring='r2',
    cv=5, 
    n_jobs=-1,
    verbose=1
)

In [18]:
# initial time
ti = time.time()

# grid-search cross-validation
gscv.fit(X_casual_train,y_casual_train)

# final time 
tf = time.time()

# time taken
print(f"time taken: {round(tf-ti,2)} sec")

Fitting 5 folds for each of 30 candidates, totalling 150 fits
time taken: 8.44 sec


In [19]:
gscv.cv_results_

{'mean_fit_time': array([0.08544669, 0.05065513, 0.08950882, 0.0464169 , 0.08651476,
        0.04669123, 0.08988228, 0.0478282 , 0.0937634 , 0.05039525,
        0.08486781, 0.05008173, 0.08280435, 0.04698787, 0.0832665 ,
        0.04659863, 0.07921047, 0.04782224, 0.13001261, 0.05332336,
        0.14572363, 0.06875739, 0.15601959, 0.0659471 , 0.43717451,
        0.07129102, 0.46129265, 0.07658792, 0.42610354, 0.07861142]),
 'std_fit_time': array([0.0090778 , 0.00252704, 0.00250256, 0.00273527, 0.00254328,
        0.00204631, 0.00213187, 0.00191189, 0.0072186 , 0.00380152,
        0.0130477 , 0.00394144, 0.0073362 , 0.00558412, 0.01055793,
        0.00472699, 0.00696865, 0.00573939, 0.01288257, 0.00432759,
        0.00993209, 0.00429532, 0.02113209, 0.00603042, 0.03142217,
        0.00836103, 0.03806208, 0.00152784, 0.09825588, 0.01444724]),
 'mean_score_time': array([0.01864924, 0.01496367, 0.02050486, 0.01482348, 0.02115459,
        0.0159164 , 0.01910644, 0.01757293, 0.01852365, 0.01

In [20]:
# list of columns to show
column_list = ['param_preprocessor__num__polynomial__degree',
               'param_preprocessor__num__polynomial__interaction_only',
               'param_lasso__alpha',
               'param_lasso__max_iter',
               'mean_test_score',
               'std_test_score',
               'rank_test_score'
              ]
# create result dataframe
result_df = pd.DataFrame(gscv.cv_results_)[column_list]

# rename columns
result_df.rename(
    columns=lambda name: name.split('__')[-1],inplace=True
)

# order by rank
result_df.sort_values(
    by='rank_test_score', ascending=True, inplace=True, ignore_index=True
)

result_df

,degree,interaction_only,alpha,max_iter,mean_test_score,std_test_score,rank_test_score
0,4,False,0.01,10000,0.817814,0.007092,1
1,4,False,0.01,5000,0.817814,0.007092,1
2,4,False,0.01,20000,0.817814,0.007092,1
3,4,False,0.1,20000,0.657246,0.006787,4
4,4,False,0.1,10000,0.657246,0.006787,4
5,4,False,0.1,5000,0.657246,0.006787,4
6,4,True,0.01,20000,0.618719,0.010540,7
7,4,True,0.01,10000,0.618719,0.010540,7
8,4,True,0.01,5000,0.618719,0.010540,7
9,4,True,0.1,20000,0.561331,0.007897,10


In [21]:
gscv.best_params_

{'lasso__alpha': 0.01,
 'lasso__max_iter': 5000,
 'preprocessor__num__polynomial__degree': 4,
 'preprocessor__num__polynomial__interaction_only': False}

In [22]:
round(gscv.best_score_,6)

0.817814

In [23]:
best_model_casual = gscv.best_estimator_
best_model_casual

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler()),
                                                                  ('polynomial',
                                                                   PolynomialFeatures(degree=4))]),
                                                  ['temp', 'humidity',
                                                   'windspeed', 'month_idx',
                                                   'hour']),
                                                 ('cat',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['holiday', 'workingday',
                                                   'day_of_week',
                                                   'weather'])])),
                ('lasso', Lasso(alpha=0.01, max_iter=5000))])

In [24]:
best_model_casual.fit(X_casual_train,y_casual_train);

In [25]:
# training score
casual_training_score = best_model_casual.score(X_casual_train, y_casual_train)

print(f'Casual Train score: {round(casual_training_score,6)}')


Casual Train score: 0.821153


**Casual Test **

In [26]:
X_casual_test,y_casual_test = test[features], test[target_variable]

In [27]:
best_model_casual.fit(X_casual_test,y_casual_test);

In [28]:
# training score
casual_test_score = best_model_casual.score(X_casual_test, y_casual_test)

print(f'Casual Test score: {round(casual_test_score,6)}')

Casual Test score: 0.822295


**Casual total prediction**

In [29]:
X_casual_total, y_casual_total = bike[features], bike[target_variable]

In [30]:
y_casual_pred = best_model_casual.predict(X_casual_total)
y_casual_pred

array([1.48057208, 0.9511494 , 0.73362157, ..., 2.27855827, 2.25470619,
       2.07337434])

**REGISTERED USERS**

In [64]:
# we need to delete either the "temp" or the "atemp" because they are highly corelated with each other.
numerical_features = [
     'temp',
     #'atemp', ## temp was better than atemp. Training score is 2 points more 
     'humidity', ## add alot like 3 points
     'windspeed', # added some more power
     'month_idx',
     'hour' 
]

categorical_features = [
    #'season', Adding season does not add much. Mostly weather matters.
    'holiday',
    'workingday',
    'day_of_week',
    'weather' # our as a categorical training score is 78.
]

features = numerical_features + categorical_features

target_variable = 'log_registered' # for registered it it aroun 20 but casual gets 45 with these features except atemp.


In [65]:
X_registered_train, y_registered_train = train[features], train[target_variable]

In [66]:
# scaling and polynomial features
numerical_transformer = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('polynomial', PolynomialFeatures())
    ]
)

In [67]:
# one-hot encoding
categorical_transformer = Pipeline(
    steps=[
        ('ohe', OneHotEncoder(drop='first'))
    ]
)

In [68]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_transformer, numerical_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)

In [69]:
estimator = Pipeline(
    steps=[
        ('preprocessor', preprocessor),   # preprocessing step
        ('lasso', Lasso()) # lasso regression
    ]
)

In [70]:
param_grid = {
    'preprocessor__num__polynomial__degree': [5],
    'preprocessor__num__polynomial__interaction_only': [False,True],
    'lasso__alpha': [100.,10.,1.,0.1,0.01],
    'lasso__max_iter': [5_000, 10_000,20_000]
}

In [71]:
gscv = GridSearchCV(
    estimator=estimator,
    param_grid=param_grid,
    scoring='r2',
    cv=5, 
    n_jobs=-1,
    verbose=1
)

In [72]:
# initial time
ti = time.time()

# grid-search cross-validation
gscv.fit(X_registered_train,y_registered_train)

# final time 
tf = time.time()

# time taken
print(f"time taken: {round(tf-ti,2)} sec")

Fitting 5 folds for each of 30 candidates, totalling 150 fits
time taken: 21.9 sec


In [75]:
gscv.cv_results_

{'mean_fit_time': array([0.18188753, 0.06311464, 0.16960883, 0.05941381, 0.18798752,
        0.06912489, 0.16857605, 0.06744261, 0.16727338, 0.06368184,
        0.18523002, 0.06133184, 0.21332521, 0.06690979, 0.18956141,
        0.06573119, 0.23535395, 0.08518906, 0.58737025, 0.0831748 ,
        0.51767044, 0.07399168, 0.41317081, 0.06874862, 3.74642735,
        0.0829433 , 3.48185797, 0.06988149, 2.91653118, 0.06716418]),
 'std_fit_time': array([0.02574138, 0.00798059, 0.01315688, 0.00869591, 0.0438662 ,
        0.02985516, 0.01164051, 0.0107916 , 0.02757965, 0.00776868,
        0.03900343, 0.02172755, 0.03562639, 0.0025017 , 0.01406282,
        0.00831953, 0.02539087, 0.01420623, 0.08263611, 0.01762485,
        0.05715271, 0.01641717, 0.01632906, 0.00747925, 0.28360133,
        0.01789504, 0.17448345, 0.00406139, 0.48877365, 0.00263163]),
 'mean_score_time': array([0.03096685, 0.018327  , 0.0355546 , 0.0217608 , 0.03843727,
        0.01862965, 0.0361742 , 0.01908584, 0.0287015 , 0.01

In [76]:
# list of columns to show
column_list = ['param_preprocessor__num__polynomial__degree',
               'param_preprocessor__num__polynomial__interaction_only',
               'param_lasso__alpha',
               'param_lasso__max_iter',
               'mean_test_score',
               'std_test_score',
               'rank_test_score'
              ]
# create result dataframe
result_df = pd.DataFrame(gscv.cv_results_)[column_list]

# rename columns
result_df.rename(
    columns=lambda name: name.split('__')[-1],inplace=True
)

# order by rank
result_df.sort_values(
    by='rank_test_score', ascending=True, inplace=True, ignore_index=True
)

result_df

,degree,interaction_only,alpha,max_iter,mean_test_score,std_test_score,rank_test_score
0,5,False,0.01,10000,0.741149,0.012144,1
1,5,False,0.01,5000,0.741149,0.012144,1
2,5,False,0.01,20000,0.741149,0.012144,1
3,5,False,0.1,20000,0.610024,0.014701,4
4,5,False,0.1,10000,0.610024,0.014701,4
5,5,False,0.1,5000,0.610024,0.014701,4
6,5,True,0.01,20000,0.465801,0.019683,7
7,5,True,0.01,10000,0.465801,0.019683,7
8,5,True,0.01,5000,0.465801,0.019683,7
9,5,True,0.1,20000,0.441790,0.017672,10


In [77]:
gscv.best_params_

{'lasso__alpha': 0.01,
 'lasso__max_iter': 5000,
 'preprocessor__num__polynomial__degree': 5,
 'preprocessor__num__polynomial__interaction_only': False}

In [78]:
round(gscv.best_score_,6)

0.741149

In [79]:
best_model_registered = gscv.best_estimator_
best_model_registered

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler()),
                                                                  ('polynomial',
                                                                   PolynomialFeatures(degree=5))]),
                                                  ['temp', 'humidity',
                                                   'windspeed', 'month_idx',
                                                   'hour']),
                                                 ('cat',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['holiday', 'workingday',
                                                   'day_of_week',
                                                   'weather'])])),
                ('lasso', Lasso(alpha=0.01, max_iter=5000))])

In [80]:
best_model_registered.fit(X_registered_train,y_registered_train);

In [81]:
# training score
registered_train_score = best_model_registered.score(X_registered_train, y_registered_train)

print(f'Registered Train score: {round(registered_train_score,6)}')


Registered Train score: 0.746643


**Registered Test**

In [82]:
X_registered_test,y_registered_test = test[features], test[target_variable]

In [83]:
best_model_registered.fit(X_registered_test,y_registered_test);

In [84]:
# training score
registered_test_score = best_model_registered.score(X_registered_test, y_registered_test)

print(f'Registered Test score: {round(registered_test_score,6)}')


Registered Test score: 0.765034


**Registered total prediction**

In [85]:
X_registered_total, y_registered_total = bike[features], bike[target_variable]

In [86]:
y_registered_pred = best_model_registered.predict(X_registered_total)
y_registered_pred

array([2.20920215, 1.1780434 , 0.92263077, ..., 5.49387229, 5.14107308,
       4.23005365])

In [87]:
magic_day = pd.DataFrame({
    "datetime": [pd.to_datetime("20230415-14:00:00")],
    "season": [1],
    "holiday": [0],
    "workingday": [0],
    "weather": [3],
    "temp": [23.3],
    "atemp": [24],
    "humidity": [76],
    "windspeed": [5],
    "casual": [""],
    "registered": [""],
    "count": [""],
}).set_index("datetime")

In [88]:
magic_day = yoda(magic_day)
magic_day

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,hour
datetime,,,,,,,,,,,,,,
2023-04-15 14:00:00,1,0,0,3,23.3,24,76,5,,,,2023,4,14


In [100]:
magic_day['month_idx'] =  magic_day["month"]

In [101]:
magic_day['day_of_week'] = magic_day.index.weekday + 1
magic_day.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,hour,month_idx,day_of_week
datetime,,,,,,,,,,,,,,,,
2023-04-15 14:00:00,1,0,0,3,23.3,24,76,5,,,,2023,4,14,4,6


In [102]:
X_magic_casual = magic_day[features]
X_magic_register = magic_day[features]

In [103]:
y_pred_casual = best_model_casual.predict(X_magic_casual)
y_pred_casual

array([4.26775084])

In [104]:
y_pred_register = best_model_registered.predict(X_magic_casual)
y_pred_register

array([4.75945651])

In [105]:
y_pred_casual_nonlog = np.exp(y_pred_casual)-1
y_pred_casual_nonlog

array([70.36095266])

In [106]:
y_pred_register_nonlog = np.exp(y_pred_register)-1
y_pred_register_nonlog

array([115.68249315])

In [107]:
magic_day_count = y_pred_casual_nonlog + y_pred_register_nonlog
magic_day_count 

array([186.04344581])